C:\Users\guilh\AppData\Local\Programs\Python\Python313\Lib\site-packages\face_recognition_models\__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


Carregando rostos conhecidos...

Cadastro concluído! 0 pessoas registradas.


FileNotFoundError: [Errno 2] No such file or directory: 'fotos_teste/teste_grupo.jpg'

Diretório de trabalho alterado para: C:\Users\guilh\Desktop\projeto_reconhecimento_facial
------------------------------
Carregando rostos conhecidos...
  - caio.jpg carregado com sucesso!
  - guilherme.jpg carregado com sucesso!
  - joao.jpg carregado com sucesso!
  - pedro.jpg carregado com sucesso!

Cadastro concluído! 4 pessoas registradas.

Iniciando análise das imagens em 'fotos_teste'...

--> Processando imagem: arq.jpg.jpeg
    Encontrados 4 rostos.
--> Processando imagem: cia.jpg.jpeg
    Encontrados 7 rostos.
--> Processando imagem: festa.jpg.jpeg
    Encontrados 3 rostos.
--> Processando imagem: orbita.jpg.jpeg
    Encontrados 4 rostos.
--> Processando imagem: resenha.jpg.jpeg
    Encontrados 5 rostos.
--> Processando imagem: roca.jpg.jpeg
    Encontrados 3 rostos.

Iniciando análise das imagens em 'fotos_teste'...

--> Processando imagem: arq.jpg.jpeg
    Encontrados 4 rostos.
    Pressione qualquer tecla na janela da imagem para ver a próxima...
--> Processando imagem: cia

Diretório de trabalho alterado para: C:\Users\guilh\Desktop\projeto_reconhecimento_facial
------------------------------
Carregando rostos conhecidos...
  - caio.jpg carregado com sucesso!
  - guilherme.jpg carregado com sucesso!
  - joao.jpg carregado com sucesso!
  - pedro.jpg carregado com sucesso!

Cadastro concluído! 4 pessoas registradas.

Iniciando análise das imagens em 'fotos_teste'...

--> Processando imagem: arq.jpg.jpeg
    Encontrados 4 rostos.
    Pressione qualquer tecla na janela da imagem para ver a próxima...
--> Processando imagem: cia.jpg.jpeg
    Encontrados 7 rostos.
    Pressione qualquer tecla na janela da imagem para ver a próxima...
--> Processando imagem: festa.jpg.jpeg
    Encontrados 3 rostos.
    Pressione qualquer tecla na janela da imagem para ver a próxima...
--> Processando imagem: orbita.jpg.jpeg
    Encontrados 4 rostos.
    Pressione qualquer tecla na janela da imagem para ver a próxima...
--> Processando imagem: resenha.jpg.jpeg
    Encontrados 5 r

In [2]:
# --- Bloco de Imports (com as novas bibliotecas) ---
import face_recognition
import cv2
import os
import numpy as np
import psycopg2
from dotenv import load_dotenv  # <-- Importa a biblioteca para ler o .env
import sys # Para parar o script se a senha não for encontrada

# -----------------------------------------------------------------
# CARREGANDO A SENHA DE FORMA SEGURA
# -----------------------------------------------------------------
load_dotenv()  # Carrega as variáveis do arquivo .env

# Lê a senha do arquivo .env que você criou
SENHA_DO_BANCO = os.getenv("DB_PASSWORD")

if not SENHA_DO_BANCO:
    print("="*50)
    print("ERRO: Senha do banco de dados não encontrada!")
    print("Verifique se você criou o arquivo '.env' e colocou DB_PASSWORD='sua_senha' nele.")
    print("="*50)
    sys.exit() # Para o script
# -----------------------------------------------------------------


def conectar_bd():
    """Conecta ao banco de dados PostgreSQL."""
    try:
        conn = psycopg2.connect(
            host="localhost",
            database="reconhecimento_db",
            user="postgres",
            password=SENHA_DO_BANCO  # <-- Usa a senha carregada de forma segura
        )
        return conn
    except psycopg2.Error as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return None

# --- FASE 1: CARREGAR ROSTOS CONHECIDOS DO BANCO DE DADOS ---
print("Conectando ao banco de dados e carregando rostos conhecidos...")

encodings_conhecidos = []
nomes_conhecidos = []

conn = conectar_bd()
if conn:
    try:
        with conn.cursor() as cur:
            # Seleciona apenas funcionários ATIVOS
            sql = "SELECT nome, encoding_facial FROM funcionarios WHERE status = 'ativo'"
            cur.execute(sql)
            
            registros = cur.fetchall()
            
            for registro in registros:
                nome = registro[0]
                encoding_binario = registro[1]
                
                # Converte os dados binários (BYTEA) de volta para um array NumPy
                # O encoding de 128 floats é salvo como 1024 bytes (128 * 8 bytes por float)
                encoding = np.frombuffer(encoding_binario, dtype=np.float64)
                
                nomes_conhecidos.append(nome)
                encodings_conhecidos.append(encoding)
                print(f"  - {nome} carregado do banco com sucesso!")
                
    except psycopg2.Error as e:
        print(f"Erro ao consultar o banco de dados: {e}")
    finally:
        if conn:
            conn.close()

print(f"\nCadastro concluído! {len(nomes_conhecidos)} pessoas ativas carregadas do banco.")
print("-" * 30)


# --- NADA MUDA DAQUI PARA BAIXO ---
# O resto do seu script (Fases 2 e 3) funciona perfeitamente como antes.

caminho_do_projeto = r"C:\Users\guilh\Desktop\projeto_reconhecimento_facial"

# Tenta mudar o diretório de trabalho. Se já estiver lá, tudo bem.
try:
    os.chdir(caminho_do_projeto)
    print(f"Diretório de trabalho definido para: {os.getcwd()}")
except FileNotFoundError:
    print(f"Aviso: Não foi possível mudar o diretório para {caminho_do_projeto}. Verifique o caminho.")
    # Continua mesmo assim, assumindo que as pastas estão acessíveis


caminho_pasta_teste = "fotos_teste"
print(f"\nIniciando análise das imagens em '{caminho_pasta_teste}'...\n")

for nome_arquivo_teste in os.listdir(caminho_pasta_teste):
    if nome_arquivo_teste.lower().endswith(('.png', '.jpg', '.jpeg')):
        
        imagem_teste_path = os.path.join(caminho_pasta_teste, nome_arquivo_teste)
        print(f"--> Processando imagem: {nome_arquivo_teste}")

        imagem_teste = face_recognition.load_image_file(imagem_teste_path)
        localizacoes_rostos = face_recognition.face_locations(imagem_teste)
        encodings_teste = face_recognition.face_encodings(imagem_teste, localizacoes_rostos)
        imagem_para_marcar = cv2.cvtColor(imagem_teste, cv2.COLOR_RGB2BGR)

        print(f"    Encontrados {len(encodings_teste)} rostos.")

        for (topo, direita, baixo, esquerda), encoding_teste in zip(localizacoes_rostos, encodings_teste):
            nome = "Desconhecido"
            distancias_faciais = face_recognition.face_distance(encodings_conhecidos, encoding_teste)
            
            if len(distancias_faciais) > 0:
                melhor_indice_match = np.argmin(distancias_faciais)
                menor_distancia = distancias_faciais[melhor_indice_match]
                
                if menor_distancia < 0.56: # Sua tolerância ajustada
                    nome = nomes_conhecidos[melhor_indice_match]
            
            cor = (0, 0, 255) if nome == "Desconhecido" else (0, 255, 0)
            cv2.rectangle(imagem_para_marcar, (esquerda, topo), (direita, baixo), cor, 3)
            cv2.putText(imagem_para_marcar, nome, (esquerda + 6, baixo - 6), cv2.FONT_HERSHEY_DUPLEX, 1.0, cor, 2)

        nome_da_janela = f"Resultado para: {nome_arquivo_teste}"
        cv2.namedWindow(nome_da_janela, cv2.WINDOW_NORMAL) 
        cv2.imshow(nome_da_janela, imagem_para_marcar)
        print("    Pressione qualquer tecla na janela da imagem para ver a próxima...")
        cv2.waitKey(0) 
        cv2.destroyAllWindows()

print("\nProcesso finalizado! Todas as imagens foram analisadas.")

Conectando ao banco de dados e carregando rostos conhecidos...
  - caio carregado do banco com sucesso!
  - guilherme carregado do banco com sucesso!
  - joao carregado do banco com sucesso!
  - pedro carregado do banco com sucesso!

Cadastro concluído! 4 pessoas ativas carregadas do banco.
------------------------------
Diretório de trabalho definido para: C:\Users\guilh\Desktop\projeto_reconhecimento_facial

Iniciando análise das imagens em 'fotos_teste'...

--> Processando imagem: arq.jpg.jpeg
    Encontrados 4 rostos.
    Pressione qualquer tecla na janela da imagem para ver a próxima...
--> Processando imagem: cia.jpg.jpeg
    Encontrados 7 rostos.
    Pressione qualquer tecla na janela da imagem para ver a próxima...
--> Processando imagem: festa.jpg.jpeg
    Encontrados 3 rostos.
    Pressione qualquer tecla na janela da imagem para ver a próxima...
--> Processando imagem: orbita.jpg.jpeg
    Encontrados 4 rostos.
    Pressione qualquer tecla na janela da imagem para ver a próxi